In [3]:
import mysql.connector,sys,traceback,csv
comlistOTC=[]
with open('E:/daily/listcompanyOTC.csv') as csvfile:
    reader=csv.reader(csvfile)
    for row in reader:
        comlistOTC.append(row[0])
comlist=[]
with open('E:/daily/listcompany.csv') as csvfile:
    reader=csv.reader(csvfile)
    for row in reader:
        comlist.append(row[0])
try:
    cnx = mysql.connector.connect(user='threela', password='123456',
                                  host='127.0.0.1',
                                  database='threela')
    cursor = cnx.cursor()
    query = ("SELECT distinct timeid FROM threela.daily_tmp where timeid > 20150731 order by timeid")
    cursor.execute(query)
    datelist=cursor.fetchall()

    for d in datelist:
        print str(d[0]) + 'is starting!'
        query='SELECT Stockid,brokerid,timeid,sum(buy) as buy,sum(sell) as sell,sum(buy*buyprice)/sum(buy) as buyprice,sum(sell*sellprice)/sum(Sell) as sellprice FROM threela.daily_tmp where timeid = %s group by Stockid,brokerid,timeid'
        data=(d[0],)
        cursor.execute(query,data)
        rows=cursor.fetchall()
        insert_daily="INSERT IGNORE INTO `threela`.`daily` (`StockId`,`BrokerId`,`TimeId`,`Buy`,`Sell`,`BuyPrice`,`SellPrice`,`Source`) VALUES (%s, %s, %s, %s, %s, %s, %s,%s)"
        data_daily_all=[]
        for row in rows:

            buyprice=0
            if row[5]==None:
                buyprice=0
            else:
                buyprice=row[5]
                
            sellprice=0
            if row[6]==None:
                sellprice=0
            else:
                sellprice=row[6]
            
            source=0
            if row[0] in comlist:
                source=1
            if row[0] in comlistOTC:
                source=2
                
            data_daily=(row[0],row[1],row[2],row[3],row[4],buyprice,sellprice,source)
            data_daily_all.append(data_daily)
        print str(d[0]) + 'is end!'
        cursor.executemany(insert_daily,data_daily_all)
        cnx.commit()
except:
    #print "Unexpected error:",sys.exc_info()[0],sys.exc_info()[1],sys.exc_info()[2]
    traceback.print_exc()
finally:
    cursor.close()
    cnx.close()

20150803is starting!
20150803is end!
20150804is starting!
20150804is end!
